# Create safety analysis segments. 

Define workspace. 

In [1]:
# Define workspace
import arcpy
import os

db = arcpy.env.workspace = r"C:\Users\AhjungKim\Documents\ArcGIS\Projects\sun_cloud_pro\sc_safety_segments_2.gdb"
arcpy.env.overwriteOutput = True


Reusable functions.

In [4]:
from arcgis.gis import GIS
gis = GIS('pro')
from arcgis.features import FeatureLayer

# save feature layer to db
def save_fl(db, url, outname):
    fl = FeatureLayer(url)
    featureset = fl.query()
    featureset.save(db, outname)

# project in_data to match the target_data

def project(in_data, target_data, out_name):

    # TODO: fix this later
    target_data = r'C:\Users\AhjungKim\Documents\ArcGIS\Projects\sun_cloud_pro\sc_safety_segments_2.gdb\dataset\PM3_For_HPMS_2_ExportFeature'

    in_sr = arcpy.Describe(in_data).spatialReference
    target_sr = arcpy.Describe(target_data).spatialReference
    datum_conversion = arcpy.ListTransformations (in_sr, target_sr)[0]



    arcpy.Project_management(
        in_dataset = in_data, 
        out_dataset = 'dataset\{}'.format(out_name),
        out_coor_system = target_sr,
        transform_method = datum_conversion)

Save the feature layers locally

In [3]:
routes_url = 'https://services6.arcgis.com/clPWQMwZfdWn4MQZ/ArcGIS/rest/services/Sun_Cloud_Routes/FeatureServer/6' 

save_fl(db, routes_url, 'sun_cloud_routes')


In [5]:
milepost = 'https://services6.arcgis.com/clPWQMwZfdWn4MQZ/arcgis/rest/services/Arizona_Highway_Performance_Monitoring_System_2020_Data_/FeatureServer/1'
save_fl(db, milepost, 'milepost')

In [6]:
throughln = 'https://services6.arcgis.com/clPWQMwZfdWn4MQZ/arcgis/rest/services/Arizona_Highway_Performance_Monitoring_System_2020_Data_/FeatureServer/41'
save_fl(db, throughln, "through_ln")

In [7]:
project('through_ln', 'sun_cloud_routes', 'through_ln_project')

### Through Lane data clean up

In [8]:
# delete nulls and 0s.
through_lane = "through_ln_project"
selected = arcpy.SelectLayerByAttribute_management(through_lane,
                                                    "NEW_SELECTION", 
                                                    "NumberOfLanes IS NULL Or NumberOfLanes = 0")
count = int(arcpy.GetCount_management(selected)[0])
print(count)
if(count>0):
    arcpy.DeleteFeatures_management(selected)


38


In [9]:
#  round up .5 lane values
codeblock = """
import math
def round_up(n):
    return math.ceil(n)"""

arcpy.management.CalculateField(through_lane,
                                "NumberOfLanes",
                                "round_up(!NumberOfLanes!)",
                                "PYTHON3",
                                codeblock)


<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\sc_safety_segments_2.gdb\\through_ln_project'>

In [10]:
# unsplit the through lane by route id and number of lanes
through_lane = 'through_ln_project'
out_fc = r"{0}\{1}_unsplit".format(db, through_lane)
print(out_fc)
fields = "RouteId;NumberOfLanes"
result = arcpy.management.UnsplitLine(through_lane, out_fc, fields)
if result:
    through_lane = out_fc


C:\Users\AhjungKim\Documents\ArcGIS\Projects\sun_cloud_pro\sc_safety_segments_2.gdb\through_ln_project_unsplit


In [11]:
# Feature vertices to points
through_lane = 'through_ln_project_unsplit'
end_pts = "{}_pts".format(through_lane)
arcpy.management.FeatureVerticesToPoints(through_lane,
                                         end_pts,
                                        "BOTH_ENDS")

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\sc_safety_segments_2.gdb\\through_ln_project_unsplit_pts'>

In [12]:
# delete points that do not intersect w/ route network
road_network = 'sun_cloud_routes'
end_pts = 'through_ln_project_unsplit_pts'

result = arcpy.SelectLayerByLocation_management(in_layer = end_pts,
                                                overlap_type = "INTERSECT",
                                                select_features = road_network,
                                                search_distance = "1 Meters", 
                                                invert_spatial_relationship='INVERT')
count = int(arcpy.GetCount_management(result)[0])
print(count)
if count>0:
    arcpy.DeleteFeatures_management(result)
# garbage collection
arcpy.management.Delete(result)

83885


<Result 'true'>

In [13]:
# add fields
temp_field = "temp"
fields=[[temp_field, "TEXT"]]
arcpy.management.AddFields("through_ln_project_unsplit_pts", fields)

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\sc_safety_segments_2.gdb\\through_ln_project_unsplit_pts'>

In [14]:
# iterate the points and find intersecting through lane
# delete if "number of lanes" values are the same
pts = "through_ln_project_unsplit_pts"
lines = "through_ln_project_unsplit"
arcpy.management.MakeFeatureLayer(lines, 'line_lyr')


with arcpy.da.UpdateCursor(pts, ["Shape@", "NumberOfLanes", temp_field], where_clause="OBJECTID < 50") as cursor:
    for row in cursor:
        me = row[1]
        arcpy.management.SelectLayerByLocation ('line_lyr', 'INTERSECT', row[0], "1 Meter", "NEW_SELECTION")
        intersects = list(r[0] for r in arcpy.da.SearchCursor('line_lyr', ["NumberOFLanes"]))
        unique = list(set(intersects))
        if len(unique)==1 and me==unique[0]:
            print("update")
            row[2]="delete"
            cursor.updateRow(row)



In [15]:
# delete the flagged features
result = arcpy.SelectLayerByAttribute_management(pts, "NEW_SELECTION", "temp = 'delete'")
count = int(arcpy.GetCount_management(result)[0])
print(count)
if count>0:
    arcpy.DeleteFeatures_management(result)
# garbage collection
arcpy.management.Delete(result)

0


<Result 'true'>

Create through lane mid point.  Use this to tranfer over the number of lanes value to the routes.

In [16]:
arcpy.management.GeneratePointsAlongLines("through_ln_project_unsplit", "throughln_mid", "PERCENTAGE", None, 50, None)

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\sc_safety_segments_2.gdb\\throughln_mid'>

### Split when number of lanes change. 

Split at # of lane changes.

In [17]:
#  split the road network at points
inFeatures = 'sun_cloud_routes'
pointFeatures = 'through_ln_project_unsplit_pts'
splitresult1 = "split_at_lane_change_1".format(pointFeatures)
searchRadius = "1 Meters"

arcpy.SplitLineAtPoint_management(inFeatures, pointFeatures, splitresult1, searchRadius)

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\sc_safety_segments_2.gdb\\split_at_lane_change_1'>

Transfer over number of lanes.

In [18]:
#spatially join
arcpy.analysis.SpatialJoin(
    target_features="split_at_lane_change_1", 
    join_features="through_ln_project_unsplit", 
    out_feature_class="split_at_lane_change_1_5", 
    join_operation="JOIN_ONE_TO_ONE", 
    join_type="KEEP_ALL", 
    field_mapping="",
    match_option="LARGEST_OVERLAP",
    search_radius="5 Feet")

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\sc_safety_segments_2.gdb\\split_at_lane_change_1_5'>

### Split at mile post.

Project the milepost.

In [19]:
project('milepost', 'sun_cloud_routes', 'milepost_project')

Split at milepost. 

In [20]:
inFeatures = 'split_at_lane_change_1_5'
pointFeatures = "milepost_project"
splitresult2 = "split_at_milepost_2".format(pointFeatures)
searchRadius = "1 Meters"

arcpy.SplitLineAtPoint_management(inFeatures, pointFeatures, splitresult2, searchRadius)

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\sc_safety_segments_2.gdb\\split_at_milepost_2'>

### Split at major intersection
major == functional class is 1-7, 11-17 per Yousef Dana

From the original routes layer (NOT the segmented route), delete non-major routes.

Create major intersection points. 

In [21]:
# create a backup of sun cloud routes
arcpy.conversion.ExportFeatures("sun_cloud_routes", "sun_cloud_routes_backup")

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\sc_safety_segments_2.gdb\\sun_cloud_routes_backup'>

In [22]:
route_raw = 'sun_cloud_routes'
sql = """functional_class BETWEEN 1 AND 7 OR functional_class BETWEEN 11 AND 17"""
selected = arcpy.SelectLayerByAttribute_management(route_raw, "NEW_SELECTION", sql, "INVERT")
count = int(arcpy.GetCount_management(selected)[0])
print (count)

if count > 0:
    arcpy.DeleteFeatures_management(selected)

0


Find the real intersections. 
Reference: unsplit the original route to find the real intersections (https://support.esri.com/en/technical-article/000025044)

In [23]:
arcpy.management.UnsplitLine(route_raw, "unsplit_route")

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\sc_safety_segments_2.gdb\\unsplit_route'>

In [24]:
# Run the Intersect tool to create points at interaction.  Note that this layer is used to create the Sunc Cloud junciton layer.
arcpy.analysis.Intersect("unsplit_route", "major_intersection", "ALL", None, "POINT")

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\sc_safety_segments_2.gdb\\major_intersection'>

In [25]:
arcpy.management.MultipartToSinglepart('major_intersection', 'major_intersec_single')

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\sc_safety_segments_2.gdb\\major_intersec_single'>

Split at major intersection.

In [26]:
# split the segmented layer at the major intersection point
inFeatures = 'split_at_milepost_2'
pointFeatures = 'major_intersec_single'
splitresult = "split_at_intersection_3"
searchRadius = "1 Meters"


arcpy.SplitLineAtPoint_management(inFeatures, pointFeatures, splitresult, searchRadius)

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\sc_safety_segments_2.gdb\\split_at_intersection_3'>

### If a segment is greater than 1 mile, generate a point at every 1 mile and split at the point. (Optional: Begin & End Point fields)

In [27]:
# add length field in miles
in_f = "split_at_intersection_3"
geom_prop = "Length_Mi LENGTH_GEODESIC"
unit = "MILES_US"
result = arcpy.management.CalculateGeometryAttributes(in_f, geom_prop, unit)
print(result)

C:\Users\AhjungKim\Documents\ArcGIS\Projects\sun_cloud_pro\sc_safety_segments_2.gdb\split_at_intersection_3


Genereate 1 mile points along the long segments.

In [28]:
# create a feature layer.  select by attribute. generate point along the line
in_f = 'split_at_intersection_3'
selected = arcpy.SelectLayerByAttribute_management(in_f,"NEW_SELECTION", "Length_Mi >1")
count = int(arcpy.GetCount_management(selected)[0])
print(count)
mile_points = 'one_mile_points'
if (count>0):
    arcpy.GeneratePointsAlongLines_management(in_f, mile_points, 'DISTANCE',
                                          Distance='1 miles')
    


1578


Split at one mile points.

In [29]:

inFeatures = 'split_at_intersection_3'
pointFeatures = 'one_mile_points'
splitresult4 = 'suncloud_safety_segments'
searchRadius = "1 Meters"

arcpy.SplitLineAtPoint_management(inFeatures, pointFeatures, splitresult4, searchRadius)

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\sun_cloud_pro\\sc_safety_segments_2.gdb\\suncloud_safety_segments'>

### Field name clean up

In [30]:
# get required fields
fields = arcpy.ListFields('sun_cloud_routes')
route_fields = [field.name for field in fields if field.required is False]


all_fields = arcpy.ListFields('suncloud_safety_segments')
req_fields = [field.name for field in all_fields if field.required is True]

keep_fields = route_fields + req_fields + ['NumberOfLanes']
print(keep_fields)

['route_id', 'functional_class', 'OBJECTID', 'Shape', 'Shape_Length', 'NumberOfLanes']


In [31]:
# arcpy.management.DeleteField ('suncloud_safety_segments', keep_fields, method = 'KEEP_FIELDS')
# # arcpy.management.DeleteField("segmented_routes", "route_id;from_measure;to_measure;functional_class;facility_type", "KEEP_FIELDS")

In [32]:
data = 'suncloud_safety_segments'
fields = arcpy.ListFields(data)
f_names = [field.name for field in fields if field.required is False]

# delete one by on due to arcpy bug
for f in f_names:
    if f not in keep_fields:
        arcpy.DeleteField_management(data, f)

Alter field alias.